In [1]:
import pandas as pd
import numpy as np
from dataset import Dataset
import warnings

# SettingWithCopyWarning 무시
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)


ds = Dataset()
JSON_KEY_PATH = "./sprintda05-soomin.json"

In [3]:
master_table = pd.read_csv('./dataset/master_last_v4.csv')
point_table = pd.read_csv('./dataset/use_point_within_signup_N_days.csv')
# candidate_table = pd.read_csv('./dataset/candidate_chosen_ratio.csv')

In [ ]:
# master_table = master_table.rename(columns={'votes_within_3d':'votes_within_3d_count','chosen_user_count':'vote_nunique_user_count'})

In [4]:
master_table = master_table.drop(columns=['Unnamed: 0'])
master_table

,user_id,total_votes,unique_days,first_vote,last_vote,active_days,cohort_day,retention_day8,gender,school_id,...,votes_within_3d,period_friend_count,retention_group,address,student_count,school_type,firstquestion_id,first_question_text,first_question_category,chosen_count
0,838023,22,5,2023-04-29,2023-05-04,5,2023-04-29,0.515152,F,271.0,...,7,0.0,mid,충청남도 천안시 서북구,491,H,121,가장 어른스러운 사람은?,['성격'],262.0
1,838466,129,7,2023-05-02,2023-06-02,31,2023-05-02,0.569444,F,369.0,...,77,63.0,top25,충청남도 아산시,578,H,298,옷이 제일 많을 거 같은 사람은?,['스타일'],342.0
2,838642,10,2,2023-04-28,2023-04-29,1,2023-04-28,0.560000,F,271.0,...,10,6.0,top25,충청남도 천안시 서북구,491,H,161,화목한 가정을 꾸릴거 같은 사람은?,['인간관계'],40.0
3,839357,212,9,2023-04-29,2023-05-26,27,2023-04-29,0.515152,M,271.0,...,114,53.0,mid,충청남도 천안시 서북구,491,H,179,책을 제일 많이 읽은 것 같은 사람은?,['지식'],173.0
4,840293,170,7,2023-05-03,2023-05-15,12,2023-05-03,0.422977,F,369.0,...,130,20.0,mid,충청남도 아산시,578,H,464,뭔가 제대로 한 번 보여줄 것 같은 사람은?,['미래'],241.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4698,1510561,475,19,2023-05-25,2023-06-13,19,2023-05-25,0.451613,M,4426.0,...,68,32.0,mid,경상남도 거제시,483,H,504,얘랑 있으면 시간이 너무 빨리가는것 같아,['일상'],37.0
4699,1511957,103,11,2023-05-25,2023-06-12,18,2023-05-25,0.451613,F,5520.0,...,23,15.0,mid,경기도 광명시,500,M,1576,죽기 전에 마지막으로 대화하고 싶은 사람은?,['인간관계'],146.0
4700,1512422,243,8,2023-05-25,2023-06-01,7,2023-05-25,0.451613,F,4426.0,...,127,11.0,mid,경상남도 거제시,483,H,604,부모님한테 가장 잘할 것 같은 친구는?,['인간관계'],51.0
4701,1512542,636,12,2023-05-25,2023-06-06,12,2023-05-25,0.451613,F,4426.0,...,167,28.0,mid,경상남도 거제시,483,H,1532,이 사람의 발냄새 만큼은 용서 된다,['일상'],86.0


In [13]:
master_table.columns

Index(['Unnamed: 0', 'user_id', 'total_votes', 'unique_days', 'first_vote',
       'last_vote', 'active_days', 'cohort_day', 'retention_day8', 'gender',
       'school_id', 'grade', 'group_id', 'votes_within_3d',
       'period_friend_count', 'retention_group', 'address', 'student_count',
       'school_type', 'firstquestion_id', 'first_question_text',
       'first_question_category', 'chosen_count'],
      dtype='object')

In [5]:
def table_merging(master_table, merge_table):
    result = master_table.merge(
    merge_table,
    on = 'user_id',
    how='left'
).copy()
    return result

## 가입 후 n일 이내 결제 유무

In [6]:
# 가입 후 7일 이내 결제 유무
date_user_table = point_table[['user_id','sign_up_date','first_vote_date','first_point_use_date','first_point_use_amount','vote_sameday_signup']]
date_user_table['sign_up_date'] = pd.to_datetime(date_user_table['sign_up_date'])
date_user_table['first_vote_date'] = pd.to_datetime(date_user_table['first_vote_date'])
date_user_table['first_point_use_date'] = pd.to_datetime(date_user_table['first_point_use_date'])

# 조건 컬럼
date_user_table['days_to_vote'] = (date_user_table['first_vote_date'] - date_user_table['sign_up_date']).dt.days
# 조건 컬럼
date_user_table['days_to_point'] = (date_user_table['first_point_use_date'] - date_user_table['sign_up_date']).dt.days
# 조건 컬럼
date_user_table['days_to_point_after_vote'] = (date_user_table['first_point_use_date'] - date_user_table['first_vote_date']).dt.days

final_master_table = table_merging(master_table, date_user_table)

## 처음 투표한 시간

In [7]:
# 처음 투표한 시간
user_table = master_table[['user_id']].copy()
user_table = user_table.merge(point_table[['user_id','first_vote_date']], on='user_id',how='left')
user_table['first_vote_hour'] = pd.to_datetime(user_table['first_vote_date']).dt.hour

final_master_table = table_merging(final_master_table, user_table[['user_id','first_vote_hour']])

## 투표 첫 날 투표 건수

In [8]:
qusetion_table = ds.votes.accounts_userquestionrecord()[['user_id','created_at','id']]
qusetion_table['day'] = pd.to_datetime(qusetion_table['created_at']).dt.floor("d")
qusetion_table = qusetion_table.drop(columns=['created_at'])

user_table['day'] = pd.to_datetime(user_table['first_vote_date']).dt.floor('d')

merged_table =pd.merge(
    user_table[['user_id','day']],
    qusetion_table,
    on = ['user_id','day'],
    how = 'left'
)

merged_table['first_voteday_question_count'] = (
    merged_table
    .groupby(['user_id', 'day'])['user_id']
    .transform('count')
)

merged_table = (
    merged_table
    .drop_duplicates(subset='user_id')
    .drop(columns=['id'])
)

final_master_table = table_merging(final_master_table, merged_table.drop(columns='day'))

## 열람 건 수 (비율)

In [ ]:
record = (
    ds.votes.accounts_userquestionrecord()
    .query('created_at < "2023-08-01"')
    .query('created_at >= "2023-04-28"')
)

has_read_table = record[['chosen_user_id','has_read','id']].copy()

has_read_table['total_voted'] =  (
    has_read_table
    .groupby(['chosen_user_id'])['id']
    .transform('count')
)
has_read_table['total_read'] = (
    has_read_table
    .groupby(['chosen_user_id'])['has_read']
    .transform('sum')
)
has_read_table = has_read_table[['chosen_user_id','total_voted','total_read']].rename(columns={'chosen_user_id':'user_id'}).drop_duplicates(subset='user_id')

fianl_master_table = table_merging(final_master_table, has_read_table)

## 투표 UNIQUE 유저 선택 수  (투표 대상 수)

In [24]:
# master-v1 통합

## 친구 수

In [25]:
# master-v1 통합

## 이성에게 받은 투표 수 (날짜 수정 필요)

In [26]:
opposite_sex_votes = pd.read_csv('./dataset/master_with_vote_stats.csv')
opposite_sex_votes = opposite_sex_votes.rename(columns={'opposite_votes_received':'opposite_sex_votes_received_count','opposite_vote_ratio':'opposite_sex_vote_received_ratio'})
final_master_table = table_merging(fianl_master_table,opposite_sex_votes.drop(columns=['gender','total_votes_received']))

## 첫 질문 유형

In [27]:
question_category = pd.read_csv('./dataset/first_question_category.txt')
fianl_master_table = table_merging(final_master_table,question_category.drop(columns='created_at'))

## 소속학교, 학년 성별

In [17]:
# Master Table 에 내용 있음

## 투표 등장 대비 선택받은 비율

In [29]:
# 투표 등장 대비 선택받은 비율
final_master_table = table_merging(final_master_table, candidate_table.drop(columns='chosen_user_id'))

# 최종본

In [16]:
import pandas as pd

In [9]:
final_master_table.to_csv('./integrated_master_table_v2.csv', encoding='UTF-8', index=False)